In [ ]:
%%javascript
$('#header').toggle();

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, Heatmap, Circle, Polyline
from ipywidgets import HTML, Label, Valid, Combobox, Dropdown, Text, Button, VBox, HBox, Box
from ipywidgets import Layout, AppLayout, GridspecLayout
from geopy.geocoders import Nominatim, BANFrance
from geopy.distance import geodesic
import numpy as np
import pandas as pd
from datetime import datetime


maps = {'Standard' : basemaps.OpenStreetMap.Mapnik}
basemap_selector_value = 'Standard'

countries_file = 'sql-pays.csv'
state = sorted(list(pd.read_csv(countries_file, sep=',', low_memory=False, header=None)[4]))

geolocator = Nominatim(user_agent="appMAP_cernicchiaro.giulia@gmail.com")
geolocator_fr = BANFrance()


glb = {'curr_locs' : {},
       'pane_heights' : ['12%', '35%', '48%'],
       'center_location' : None,
       'log_rows' : []
      }


header = HTML("<h2>GPS à Vol d'Oiseaux</h2>", layout=Layout(height='auto'))
header.style.text_align='center'

state_selector = Dropdown( options = list(state), layout=Layout(width='auto'), value = 'France' )

subheader_1 = HTML(
    value='<h3>Adresse choisie</h3>',
    layout=Layout(width='auto', height='auto')
)

user_search = Combobox(placeholder='Saisir une adresse et cherhcer OU cliquer sur la carte', options=[])
user_search.layout.width = '90%'

user_action = Button(description='Chercher', button_style='info', tooltip="Geolocaliser l'adresse saisie")

subheader_2 = HTML(
    value='<h4>Adresses Trouvées</h4>',
    layout=Layout(width='auto', height='auto')
)

address_selector = Dropdown()
address_selector.layout.width = '98%'

main_map = Map(basemap=basemaps.OpenStreetMap.Mapnik)
main_map.layout.height = 'auto'
center_map = geolocator.geocode(state_selector.value)
main_map.center = (center_map.latitude, center_map.longitude)
main_map.zoom = 6

circle = Circle()
circle_radius = Circle()

subheader_3 = HTML(
    value='<h3>Verifier une Adresse</h3>',
    layout=Layout(width='auto', height='auto')
)

user_verify = Combobox(placeholder="Saisir une adresse à verifier à vol d'oiseau", options=[])
user_verify.layout.width = '90%'

user_action_2 = Button(description='Verifier', button_style='info', tooltip="")

out = HTML(
    value='',
    layout=Layout(width='auto', height='auto')
)


grid_header = GridspecLayout(1, 4, height='auto')
grid_header[0, :3] = header
grid_header[0, 3] = VBox([Label("Pays : "),state_selector])

vbox_header = VBox([grid_header,
                    subheader_1,
                    HBox([user_search,user_action]),
                   ])

vbox_footer = VBox([out])


def add_log(msg):
    max_rows = 15
    glb['log_rows'].append(msg)
    if len(glb['log_rows']) > max_rows:
        glb['log_rows'].pop(0)
    out_rows=glb['log_rows'][:]
    out_rows.reverse()
    return '<h3>Activity log</h3><ul>{}</ul>'.format('<li>'.join([''] + out_rows))

def result_source(inl,inl_fr):
    if inl and inl_fr :
        out = {l.address:l for l in inl if l.address}
        out_fr = {l.address:l for l in inl_fr if l.address}
        if len(out)+len(out_fr) == 0:
            out_type = None
        elif len(out) == 0:
            out_type = 'BAN'
        elif len(out_fr) == 0:
            out_type = 'noBAN'
        else:
            out_type = 'mixte'
    elif inl_fr :
        out = {}
        out_fr = {l.address:l for l in inl_fr if l.address}
        if len(out_fr) == 0:
            out_type = None
        else:
            out_type = 'BAN'
    elif inl :
        out = {l.address:l for l in inl if l.address}
        out_fr = {}
        if len(out) == 0:
            out_type = None
        else:
            out_type = 'noBAN'
    else :
        out = {}
        out_fr = {}
        out_type = None
    return out,out_fr,out_type

def search_address(locs,locs_fr,from_button) :
    glb['curr_locs'] = {}
    msg_add = ' -> '
    ladd,ladd_fr,loc_type = result_source(locs,locs_fr)
    if loc_type:
        if loc_type == 'BAN':
            curr_state = 'France'
            glb['curr_locs'].update(ladd_fr)
        else :
            srch_country = set( [l.raw['address']['country'] for l in ladd.values()] )
            if state_selector.value in srch_country:
                curr_state = state_selector.value
            elif loc_type == 'mixte' and 'France' in srch_country:
                curr_state = 'France'
            elif len(srch_country) == 1 :
                curr_state = list(srch_country)[0]
            else :
                curr_state = ladd[list(ladd.keys())[0]].raw['address']['country']
            if curr_state == 'France' :
                glb['curr_locs'].update(ladd_fr)
                glb['curr_locs'].update({k:v for k,v in ladd.items() if v.raw['address']['country'] == curr_state and k not in ladd_fr.keys()})
                glb['curr_locs'].update({k:v for k,v in ladd.items() if v.raw['address']['country'] != curr_state and k not in ladd_fr.keys()})
            else:
                glb['curr_locs'].update({k:v for k,v in ladd.items() if v.raw['address']['country'] == curr_state and k not in ladd_fr.keys()})
                glb['curr_locs'].update({k:v for k,v in ladd.items() if v.raw['address']['country'] != curr_state and v.raw['address']['country'] != 'France' and k not in ladd_fr.keys()})
                glb['curr_locs'].update({k:v for k,v in ladd.items() if v.raw['address']['country'] == 'France' and k not in ladd_fr.keys()})
                glb['curr_locs'].update(ladd_fr)                
        address_selector.options = list(['']+list(glb['curr_locs'].keys()))
        if curr_state in state_selector.options :
            state_selector.value = curr_state
        else :
            out.value = add_log('( %s [WARNING] ) %s : pays non disponible dans la liste \n' % (datetime.now(), curr_state))
        first_loc = glb['curr_locs'][list(glb['curr_locs'].keys())[0]]
        circle.location = (first_loc.latitude, first_loc.longitude)
        circle.radius = 1
        circle.color = "blue"
        main_map.layers = [basemap_to_tiles(maps[basemap_selector_value]), circle]
        circle_radius.location = (0, 0)
        circle_radius.radius = 0
        if len(glb['curr_locs']) == 1 :
            vbox_header.children = tuple(list(vbox_header.children)[:3])
            user_search.placeholder = 'Saisir une adresse et cherhcer OU cliquer sur la carte'
            user_search.value = first_loc.address
            msg_add += 'Adresse unique trouvée : ' + str(first_loc.address)
        elif len(glb['curr_locs']) > 1 :
            vbox_header.children = tuple(list(vbox_header.children)[:3] + [subheader_2, Box([address_selector])])
            user_search.placeholder = 'Choisir une adresse parmi celles trouvées OU Chercher une autre adresse'
            user_search.value = ''
            for l in [v for k,v in glb['curr_locs'].items() if k != list(glb['curr_locs'].keys())[0]] :
                main_map.add_layer(Circle(location=(l.latitude, l.longitude),radius=1,color="blue"))
            msg_add += str(len(glb['curr_locs'])) + ' adresses trouvées'
        if loc_type == 'BAN':
            msg_add += ' ( Source : BANFrance )'
        elif loc_type == 'noBAN' :
            msg_add += ' ( Source : OpenStreetMap )'
        elif curr_state == 'France' :
            msg_add += ' ( Sources : BANFrance & OpenStreetMap )'
        else:
            msg_add += ' ( Sources : OpenStreetMap & BANFrance )'
        user_action.description = 'Valider'
        vbox_footer.children = tuple([list(vbox_footer.children)[-1]])
    else :
        address_selector.options = list([''])
        vbox_header.children = tuple(list(vbox_header.children)[:3])
        if from_button :
            src_plch = 'pour ' + user_search.value
        else :
            src_plch = "à l'endroit cliqué"
        user_search.placeholder = 'Aucun resultat ' + src_plch + '. Ressayer'
        user_search.value = ''
        msg_add += '[ERROR] Aucune adresse trouvée'
        if locs or locs_fr:
            msg_add += ' ( Resultats sans adresse :'
            if locs :
                msg_add += ' BANFrance'
            if locs_fr :
                msg_add += ' OpenStreetMap'
            msg_add += ' )'
    return msg_add


def on_state_selected(change):
    if (geolocator.geocode(state_selector.value, language='fr')):
        center_map = geolocator.geocode(state_selector.value)
        main_map.center = (center_map.latitude, center_map.longitude)
        main_map.zoom = 6
        msg = '( %s ) Pays selectioné : %s \n' % (datetime.now(), state_selector.value)    
    else:
        msg = '( %s [ERROR] ) %s : pays non disponible \n' % (datetime.now(), state_selector.value)    
    out.value = add_log(msg)

def on_text_entry(change):
    if user_search.value != '' :
        if user_search.value not in address_selector.options :
            user_action.description = 'Chercher'
            msg = '( %s ) Nouvelle recherche \n' % (datetime.now())    
        else :
            user_action.description = 'Valider'
            msg = '( %s ) Recherche à valider \n' % (datetime.now())    
        #out.value = add_log(msg)

def on_button_clicked(b):
    if user_search.value != '' :
        user_search.options = tuple(list(set(list(user_search.options) + [user_search.value])))
        if user_action.description == 'Chercher':
            msg = '( %s ) Adresse saisie : %s \n' % (datetime.now(), user_search.value)
            msg += search_address(geolocator.geocode(user_search.value,exactly_one=False,limit=100,addressdetails=True,language='fr'),
                                  geolocator_fr.geocode(user_search.value,exactly_one=False,limit=10),
                                  True
                                 )
        elif user_action.description == 'Valider':
            user_verify.options = tuple(list(set(list(user_verify.options) + [user_search.value])))
            msg = '( %s ) Adresse validée : %s \n' % (datetime.now(), user_search.value)    
            found_address = glb['curr_locs'][user_search.value]
            glb['curr_locs'] = {}
            address_selector.options = list([''])
            circle.location = (found_address.latitude, found_address.longitude)
            circle.radius = 1
            circle.color = "blue"
            main_map.layers = [basemap_to_tiles(maps[basemap_selector_value]), circle]
            circle_radius.location = (found_address.latitude, found_address.longitude)
            circle_radius.radius = 100000
            circle_radius.color = "red"
            circle_radius.fill_color = "green"
            main_map.add_layer(circle_radius)
            center_map = found_address
            main_map.center = (center_map.latitude, center_map.longitude)
            main_map.zoom = 8
            vbox_header.children = tuple(list(vbox_header.children)[:3] + [HBox([Valid(value=True),Label(found_address.address+' : SELECTIONNEE. Descendre sous la carte pour voir les destinations.')])])
            user_search.placeholder = 'Saisir une adresse et cherhcer OU cliquer sur la carte'
            user_search.value = ''
            user_action.description = 'Chercher'
            vbox_footer.children = tuple([subheader_3,
                                          HBox([Valid(value=True,description='Adresse'),Label(found_address.address)]),
                                          HBox([user_verify,user_action_2]),
                                          out])
            glb['center_location'] = found_address
        out.value = add_log(msg)

def handle_interaction(**kwargs):
    if kwargs['type'] == 'click':
        if user_action.description == 'Valider' and str(kwargs['coordinates'][0])+', '+str(kwargs['coordinates'][1]) in [str(glb['curr_locs'][k].latitude)+', '+str(glb['curr_locs'][k].longitude) for k in list(glb['curr_locs'].keys())] :
            zoom_loc = 18
            zoom_loc_str = 'adresse'
        else :
            if main_map.zoom < 4:
                zoom_loc = 4
                zoom_loc_str = 'pays'
            elif main_map.zoom < 6:
                zoom_loc = 7
                zoom_loc_str = 'region'
            elif main_map.zoom < 8:
                zoom_loc = 9
                zoom_loc_str = 'departement'
            elif main_map.zoom < 12:
                zoom_loc = 15
                zoom_loc_str = 'commune'
            else :
                zoom_loc = 18
                zoom_loc_str = 'adresse'
        msg = '( %s ) Coordonées choisies : %s ( zoom map : %s - zoom crd : %s )\n' % (datetime.now(), str(kwargs['coordinates'][0])+', '+str(kwargs['coordinates'][1]), str(round(main_map.zoom)), zoom_loc_str)    
        msg += search_address(geolocator.reverse(str(kwargs['coordinates'][0])+', '+str(kwargs['coordinates'][1]),exactly_one=False,addressdetails=True,zoom=zoom_loc,language='fr'),
                              geolocator_fr.reverse(str(kwargs['coordinates'][0])+', '+str(kwargs['coordinates'][1]),exactly_one=False),
                              False
                             )
        if user_search.value != '' :
            user_search.options = tuple(list(set(list(user_search.options) + [user_search.value])))
            user_verify.options = tuple(list(set(list(user_verify.options) + [user_search.value])))            
        out.value = add_log(msg)

def on_address_selected(change):
    user_search.value = address_selector.value
    if user_search.value == '' :
        msg = '( %s ) Adresse vide \n' % (datetime.now()) 
    else :
        msg = '( %s ) Adresse selectionnée : %s \n' % (datetime.now(), address_selector.value) 
    #out.value = add_log(msg)

def on_action_change(change):
    if user_action.description == 'Chercher':
        user_action.tooltip = "Geolocaliser l'adresse saisie"
    else:
        user_action.tooltip = "Selectionner l'adresse et construir le rayon"
    msg = '( %s ) Tooltip actualisé\n' % (datetime.now())    
    #out.value = add_log(msg)

def on_verify_clicked(b) :
    user_search.options = tuple(list(set(list(user_search.options) + [user_verify.value])))
    locs = geolocator.geocode(user_verify.value,exactly_one=False,limit=100,addressdetails=True,language='fr') #,country_codes=glb['center_location'].raw['address']['country_code'])
    locs_fr = geolocator_fr.geocode(user_verify.value,exactly_one=False,limit=10)
    ladd,ladd_fr,loc_type = result_source(locs,locs_fr)
    if 'properties' in glb['center_location'].raw.keys() :
        loc_origin = 'BAN'
    elif 'address' in glb['center_location'].raw.keys() :
        loc_origin = 'OSM'
    else :
        loc_origin = None
    msg = '( %s ) Adresse à verififer : %s \n' % (datetime.now(), user_verify.value)
    if loc_type :
        if loc_origin:
            if loc_type == 'BAN':
                cur_loc = locs_fr[0]
                len_loc = len(ladd_fr)
            elif loc_type == 'noBAN':
                cur_loc = locs[0]
                len_loc = len(ladd)
            else:
                if loc_origin == 'BAN' or glb['center_location'].raw['address']['country'] == 'France':
                    cur_loc = locs_fr[0]
                else:
                    cur_loc = locs[0]
                len_loc = len(ladd) + len(ladd_fr)
            user_verify.value = cur_loc.address
            user_search.options = tuple(list(set(list(user_search.options) + [user_verify.value])))
            user_verify.options = tuple(list(set(list(user_verify.options) + [user_verify.value])))
            if len_loc == 1 :
                msg += ' -> trouvée unique'
                vbox_footer.children = tuple(list(vbox_footer.children)[:3] + [list(vbox_footer.children)[-1]])
            elif len_loc > 1 :
                msg += ' -> ' + str(len_loc) + ' trouvées, premiere'
                vbox_footer.children = tuple(list(vbox_footer.children)[:3] + [Label(str(len_loc) + ' adresses trouvées avec cette recherche, utiliser le panneau de recherche en haut pour explorer les resultats')] + [list(vbox_footer.children)[-1]])
            current_distance = geodesic((cur_loc.latitude, cur_loc.longitude),(glb['center_location'].latitude, glb['center_location'].longitude)).km
            if current_distance <= 100 :
                msg += ' dans le rayon'
                color_result = 'green'
                vbox_footer.children = tuple(list(vbox_footer.children)[:-1] + [HBox([Valid(value=True),Label('Accessible : ' + str(current_distance) + " km à vol d'oiseau")])] + [list(vbox_footer.children)[-1]])
            else:
                msg += ' hors rayon'
                color_result = 'red'
                vbox_footer.children = tuple(list(vbox_footer.children)[:-1] + [HBox([Valid(value=False),Label('Inaccessible : ' + str(current_distance) + " km à vol d'oiseau")])] + [list(vbox_footer.children)[-1]])
            main_map.layers = [basemap_to_tiles(maps[basemap_selector_value]), circle, circle_radius]
            main_map.add_layer(Circle(location=(cur_loc.latitude, cur_loc.longitude),radius=1,color=color_result))
            main_map.add_layer(Polyline(locations=[[cur_loc.latitude, cur_loc.longitude],[glb['center_location'].latitude, glb['center_location'].longitude]],fill=False,color=color_result))
        else:
            msg += ' [Erreur inattendu]'
    else :
        msg += ' -> non trouvée'
        if locs or locs_fr:
            msg += ' ( Resultats sans adresse :'
            if locs :
                msg += ' BANFrance'
            if locs_fr :
                msg += ' OpenStreetMap'
            msg += ' )'
        user_verify.placeholder = 'Aucun resultat pour ' + user_verify.value + '. Ressayer'
        user_verify.value = ''
        vbox_footer.children = tuple(list(vbox_footer.children)[:3] + [list(vbox_footer.children)[-1]])
    out.value = add_log(msg)


state_selector.observe(on_state_selected, names='value')
user_search.observe(on_text_entry,names='value')
user_action.on_click(on_button_clicked)
user_action.observe(on_action_change,names='description')
address_selector.observe(on_address_selected,names='value')
main_map.on_interaction(handle_interaction)
user_action_2.on_click(on_verify_clicked)


AppLayout(header=vbox_header,
          center=main_map,
          footer=vbox_footer,
          pane_heights=glb['pane_heights'],
          height='2000px',
          grid_gap="1px")